In [1]:
from pathlib import Path
from astropy.io import fits
from astropy.table import Table
import shutil


source=Path("/home/nicolas/Documents/Research/PhD/JWST-Data/JWST Data/PRISM/Spectra1D/")

mast_folder=source / "MAST"
jades_folder=source / "JADES"


This search in both folders that any of the files contains the same sequence of numbers

In [ ]:
obj_ID="6002"

print("Example 1: \n")
for file in jades_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)
    
for file in mast_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)    



obj_ID="5113"

print("Example 2: \n")

for file in jades_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)
    
for file in mast_folder.iterdir():
    if obj_ID in file.name:
        print(file.name)    
        


Example 1: 

hlsp_jades_jwst_nirspec_goods-s-deephst-00006002_clear-prism_v1.0_x1d.fits
Example 1: 

hlsp_jades_jwst_nirspec_goods-s-deephst-10005113_clear-prism_v1.0_x1d.fits
jw01210-o001_s000005113_nirspec_clear-prism_x1d.fits


This extracts the obj_ID from JADES folder files, but only works for the ones starting in 000

In [3]:
import re

for file in jades_folder.iterdir():
    match=re.search(r"100(\d+)", file.name)
    if match:
        ID = match.group(1)
        print(ID)

05469
15338
58975
12477
37201
05113
14220
00183
08071
40620
36262
35328
13597
00098
12081
01587
11955
08722
00633
13393
12005
36834
05447
14117
09609
13704
01568
13774
35295
00367
13545
73
01283
03811
11378
40
00796
38972
12284
04398
06084
09506
00814
12552
13682
13609
08231
12655
09382
12344
01946
37047
12566
15193
10642
11388
00966
10639
13578
09848
15344
11974
12488
13385
14177
00081
01843
00625
09320
03788
16442
12669
09693
12616
10770
11987
05217
12194
10691
11980
00626
00219
01761
56849
36017
11849
03750
13620
04819
04721
40868
13618
12493
00338
13797
16186
01697
00618
16374
00085
01916
13905
01892
00483
00478
14170
08067
12170
00160
05367
04061
12511


Let's now combine them: extract all the IDs from JADEs and then search for coincidences in MAST files

In [4]:
import re
import numpy as np

#Extract IDs and plug them into array
IDs_array = []


for file in jades_folder.iterdir():
    match=re.search(r"deephst-100(\d+)", file.name)
    if match:
        obj_ID = int(match.group(1))
        IDs_array.append(obj_ID)

        
IDs_array.sort()

#Print files that contain ID in their names

jades_files=[]
mast_files=[]

for obj_ID in IDs_array:
    for file in jades_folder.iterdir():
        if str(obj_ID) in file.name:
            jades_files.append(file.name)
        
    for file in mast_folder.iterdir():
        if str(obj_ID) in file.name:
            mast_files.append(file.name)    
            
#Compare lengths

print(len(IDs_array),
len(jades_files),
len(mast_files) )


110 140 151


I don't know why there are more entries in jades_files than in IDs_array, they should be the same size. Let's just extract the ID and then us the array to make the plots with a loop

In [11]:
import re
import numpy as np
import matplotlib.pyplot as plt



#Extract IDs and plug them into array
IDs_array = []


for file in jades_folder.iterdir():
    match=re.search(r"deephst-100(\d+)", file.name)
    if match:
        obj_ID = match.group(1)
        IDs_array.append(obj_ID)     

 

This is the function that we will use in the loop, it plots the spectra

In [26]:
def plot_spectrum(MAST_file, JADES_file, ID):
  
    
    # ~ fits.open('file.fits')
    t_MAST=Table.read(MAST_file, hdu="EXTRACT1D")
    w_MAST=t_MAST['WAVELENGTH']
    flux_MAST=t_MAST['FLUX'] #flux in Jy
    #z=1
    #rest_w=w/(1+z) #in micras
    #rest_w_Angstrom=rest_w*1e-6/1e-10 # rest wavelength in Angstrom
    w_Angstrom=w_MAST*1e-6/1e-10 # observed wavelength in Angstrom
    flux_ergs = 2.99792458E-05 *flux_MAST / w_Angstrom**2. #flux in erg s^-1 cm^-2 A^-1



    #archivo de JADES
    t_JADES=Table.read(JADES_file, hdu="EXTRACT1D")
    w_JADES=t_JADES['WAVELENGTH']
    flux_JADES=t_JADES['FLUX'] #flux in Jy

    
    plt.rcParams["figure.figsize"] = (8, 6)
    plt.title(f'ID {ID}, z=', fontsize=18.)
    
    plt.step(w_MAST/1e-4,flux_ergs, label='MAST Data-Archive', color='Blue')
    plt.step(w_JADES/1e-4,flux_JADES, label='JADES Survey', color='Green')
    plt.xlabel(r'$\lambda_{obs}$ [$\AA$]', fontsize=22)
    plt.ylabel(r'FLUX [erg s$^{-1}$ cm$^{-2}$ $\AA^{-1}$]', fontsize=22)
    legend=plt.legend(loc=4,labelspacing=0.1)
    legend=plt.legend(loc='best',labelspacing=0.1)
    plt.setp(legend.get_texts(),fontsize='14')
    plt.savefig(f"PRISM_Plots/{ID}.pdf")
    plt.close()
    

Test of the loop

In [ ]:
for ID in ["00367", "01587", "01697"]:
    mast_file = './Spectra1D/MAST/jw01210-o001_s0000'+ID+'_nirspec_clear-prism_x1d.fits'
    jades_file = './Spectra1D/JADES/hlsp_jades_jwst_nirspec_goods-s-deephst-100'+ID+'_clear-prism_v1.0_x1d.fits'

    plot_spectrum(mast_file, jades_file, ID)

Finally, just change the array [---] for IDs_array computed previously